In [1]:
!pip3 install --upgrade --user bigframes==0.25 \
tree-sitter==0.20.1 \
loguru==0.7.2 \
chromadb==0.4.22 \
google-cloud-aiplatform==1.45.0 \
langchain_google_vertexai==0.1.2 \
langchain==0.1.12 \
gradio==3.41.2 \
--quiet


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [2]:
import sys


PROJECT_ID = "my-project-6125-429500"
LOCATION = "us-central1"

import warnings
warnings.filterwarnings("ignore")

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)


In [3]:
import os

tree_sitter_parent_dir_name = "tmp"
tree_sitter_parent_dir = os.path.join(os.getcwd(), tree_sitter_parent_dir_name)

!git clone https://github.com/tree-sitter/tree-sitter-python.git {tree_sitter_parent_dir}/tree-sitter-python/

# Clone your code repository, replace with the repository you want to use
github_repository_clone_url = "https://github.com/google/python-fire.git"
!git clone {github_repository_clone_url}


Cloning into '/home/jupyter/tmp/tree-sitter-python'...
remote: Enumerating objects: 3212, done.
remote: Counting objects: 100% (1389/1389), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 3212 (delta 1196), reused 1267 (delta 1149), pack-reused 1823
Receiving objects: 100% (3212/3212), 21.70 MiB | 27.95 MiB/s, done.
Resolving deltas: 100% (1976/1976), done.
Cloning into 'python-fire'...
remote: Enumerating objects: 2193, done.
remote: Counting objects: 100% (1060/1060), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 2193 (delta 845), reused 862 (delta 762), pack-reused 1133
Receiving objects: 100% (2193/2193), 6.14 MiB | 24.74 MiB/s, done.
Resolving deltas: 100% (1446/1446), done.


In [5]:
!pip install tree-sitter==0.20.1


In [3]:
import os

tree_sitter_parent_dir_name = "tmp"
tree_sitter_parent_dir = os.path.join(os.getcwd(), tree_sitter_parent_dir_name)

# Clone the tree-sitter-python grammar
!git clone https://github.com/tree-sitter/tree-sitter-python.git {tree_sitter_parent_dir}/tree-sitter-python/


fatal: destination path '/home/jupyter/tmp/tree-sitter-python' already exists and is not an empty directory.


In [4]:
from tree_sitter import Language

# Build Tree sitter Parser object
Language.build_library(
    f"{tree_sitter_parent_dir}/my-languages.so",
    [
        f"{tree_sitter_parent_dir}/tree-sitter-python",
    ],
)


True

In [1]:
import os
import glob
import dataclasses
import uuid
from loguru import logger
from tree_sitter import Language, Parser
from langchain_google_vertexai.embeddings import VertexAIEmbeddings
import chromadb
import pandas as pd
from langchain.vectorstores import Chroma

# Define directories
tree_sitter_parent_dir_name = "tmp"
tree_sitter_parent_dir = os.path.join(os.getcwd(), tree_sitter_parent_dir_name)

# Define the repository clone URL
github_repository_clone_url = "https://github.com/google/python-fire.git"

PERSISTENT_DISK_DIR = "chromadb"
collection_name = "bug_fix_suggestions"

@dataclasses.dataclass
class Snippet:
    """Dataclass for storing Embedded Snippets"""
    id: str
    embedding: list[float] | None
    snippet: str
    filename: str
    language: str

class CodeParser:
    """Code Parser Class."""

    def __init__(self, language: str, node_types: list[str], path_to_object_file: str):
        self.node_types = node_types
        self.language = language
        try:
            self.parser = Parser()
            self.parser.set_language(
                Language(f"{path_to_object_file}/my-languages.so", language)
            )
        except Exception as e:
            logger.exception("failed to build %s parser: ", e)

    def parse_file(self, content: str, filename: str):
        try:
            tree = self.parser.parse(content)
        except Exception as e:
            logger.error(f"Failed to parse snippet: {filename} \n Error: {e}")
            return []

        cursor = tree.walk()
        parsed_snippets = []

        while cursor.goto_first_child():
            if cursor.node.type in self.node_types:
                parsed_snippets.append(
                    Snippet(
                        id=str(uuid.uuid4()),
                        snippet=cursor.node.text,
                        filename=filename,
                        language=self.language,
                        embedding=None,
                    )
                )

            while cursor.goto_next_sibling():
                if cursor.node.type in self.node_types:
                    parsed_snippets.append(
                        Snippet(
                            id=str(uuid.uuid4()),
                            snippet=cursor.node.text,
                            filename=filename,
                            language=self.language,
                            embedding=None,
                        )
                    )
        return parsed_snippets

    def parse_directory(self, code_directory_path):
        parsed_contents = []
        for filename in glob.glob(f"{code_directory_path}/**/*.py", recursive=True):
            with open(filename, "rb") as codefile:
                code_content = codefile.read()

            parsed_content = self.parse_file(code_content, filename)
            parsed_contents.extend(parsed_content)

        return parsed_contents

def to_dataframe_row(embedded_snippets: list[Snippet]):
    outputs = []
    for embedded_snippet in embedded_snippets:
        output = {
            "ids": embedded_snippet.id,
            "embeddings": embedded_snippet.embedding,
            "snippets": embedded_snippet.snippet,
            "metadatas": {
                "filenames": embedded_snippet.filename,
                "languages": embedded_snippet.language,
            },
        }
        outputs.append(output)
    return outputs

# Parsing python files from the local code repository
parser = CodeParser(
    language="python",
    node_types=["class_definition", "function_definition"],
    path_to_object_file=tree_sitter_parent_dir,
)

logger.info("Parsing Code...")
parsed_snippets = parser.parse_directory(
    github_repository_clone_url.split("/")[-1].split(".")[0]
)

logger.info("Generating Code Embeddings...")
# Generate embedding for the parsed code snippet blocks
embedder = VertexAIEmbeddings(model_name="textembedding-gecko@003")
snippet_texts = list(map(lambda x: x.snippet.decode("ISO-8859-1"), parsed_snippets))
embedded_texts = embedder.embed_documents(texts=snippet_texts)

embedded_snippets = []
for code_text, embedding, snippet in zip(
    snippet_texts, embedded_texts, parsed_snippets
):
    snippet.snippet = code_text
    snippet.embedding = embedding
    embedded_snippets.append(snippet)

# Convert Snippets to DataFrame for ChromaDB Ingestion
data = pd.DataFrame(to_dataframe_row(embedded_snippets))

client = chromadb.PersistentClient(path=PERSISTENT_DISK_DIR)
collection = client.get_or_create_collection(
    name=collection_name, metadata={"hnsw:space": "cosine"}
)
logger.info(
    f"Adding {data.shape[0]} Code snippets and embedding to "
    "local chroma db collection..."
)
collection.add(
    documents=data["snippets"].tolist(),
    embeddings=data["embeddings"].tolist(),
    metadatas=data["metadatas"].tolist(),
    ids=data["ids"].tolist(),
)

client = None
logger.info("Code Ingestion Completed.")

# Create langchain Chroma instance for downstream use
chroma = Chroma(
    persist_directory=PERSISTENT_DISK_DIR,
    collection_name=collection_name,
    embedding_function=embedder,
)


2024-07-22 20:55:56.486 | INFO     | __main__:<module>:113 - Parsing Code...
2024-07-22 20:55:56.581 | INFO     | __main__:<module>:118 - Generating Code Embeddings...
I0000 00:00:1721681756.592340   12795 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
2024-07-22 20:55:59.411 | INFO     | __main__:<module>:139 - Adding 316 Code snippets and embedding to local chroma db collection...
2024-07-22 20:55:59.993 | INFO     | __main__:<module>:151 - Code Ingestion Completed.


In [2]:
%%writefile prompts.yaml

detect_language:
  contextual: >
    You are an elite programmer who knows syntax to multiple programming language.
    You are provided with a code snippet encapsulated by back-ticks (```).
    You are tasked to identify in what programming language out of python, java, golang the code snippet belongs to.
    Only give the name of the language and nothing else in the output.
    ```
    {text}
    ```

  natural_language: >
    Given a piece of natural language text, determine if any of the programming languages "go", "java", or "python" is mentioned.
    ### Text:
    {text}

    If it's not evident from the text, the output should strictly be one of: go, java, python, or None


bug_detection:
  model:
    name: "code-bison-32k@002"
    temperature: 0
    max_output_tokens: 1024
    prefix: ""
  prompts:
    zero_shot: >
      You are an expert programmer.
      Identify the bug in the following code snippet and suggest a fix.

      Considering ### Task, respond with code that works in an IDE only.
      Respond only with {language} code.
      Include code comments explaining what steps you are taking.

      Do not use any textual explanations.
      Do not return Markdown.
      Do not emit ```{language}
      Do not emit ```

      ### Task
      Identify and fix the bug in the snippet below:

      ```

      {text}

      ```

      Reference this code snippet as context when completing the ### Task.

      ### Context: \n

      ```

      {context}

      ```

      Note: Please include necessary library imports. Ensure that all used variables and arguments are available.


fix_suggestion:
  model:
    name: "gemini-1.0-pro-001"
    temperature: 0.1
    max_output_tokens: 1024
    prefix: ""
  prompts:
    zero_shot: >
      Consider yourself a highly skilled programming expert with a deep understanding of various programming languages and paradigms.
      Your current task is to provide the most efficient and elegant code solution possible for the given user query.
      Leverage any provided context code if relevant, or seamlessly craft new code when necessary.

      ###User_Query: \n

      ```

      {text}

      ```

      ###Context_Code: \n

      ```

      {context}

      ```

      Instructions:
      1. Carefully understand the programming task described in the ###User_Query.
      2. Thoroughly assess the Context Code. Determine if it contains functions, structures, or logic that directly align with the ###User_Query.
      3. If the ###Context_Code contains a function or class that already implements the desired functionality, your task is to use that function or class directly in your generated code.
         Do not create new implementations unless necessary.
      4. If the ###Context_Code has helpful parts that you can modify for the task, adapt and use those parts in your solution.
      5. If the ###Context_Code is irrelevant or insufficient, generate a new, complete code solution from scratch that fulfills the User Query.


Overwriting prompts.yaml


In [3]:
import yaml
from abc import ABC
from typing import Dict, Any
from pydantic import BaseModel
from langchain.prompts import PromptTemplate
from langchain_google_vertexai import VertexAI

class AssistantResponse(ABC, BaseModel):
    class Config:
        arbitrary_types_allowed = True

    input_query: str
    prompt: str
    model_name: str
    model_parameters: Dict[str, Any]
    intent: str
    response: str

    def to_dict(self) -> dict[str, Any]:
        return self.dict()

class PairProgrammingAssistant:
    def __init__(self, chroma: Chroma, prompt_tag: str):
        self.config = self.read_config("prompts.yaml")
        self.chroma = chroma
        self.prompt_tag = prompt_tag
        self.model_name = None
        self.intent = None
        self.temperature = 0.2
        self.max_output_tokens = 1024

    @staticmethod
    def read_config(config_file: str):
        with open(config_file, "r", encoding="utf-8") as stream:
            config = yaml.safe_load(stream)
        return config

    def detect_language(self, text: str):
        language_detect_prompt = None
        if self.intent == "bug_detection":
            language_detect_prompt = self.config["detect_language"]["contextual"]
        elif self.intent == "fix_suggestion":
            language_detect_prompt = self.config["detect_language"]["natural_language"]

        logger.info(f"Language detection prompt: {language_detect_prompt}")
        llm = self.get_llm_model()
        language = llm.predict(language_detect_prompt.format(text=text))
        return language.strip().lower()

    def get_model_parameters(self):
        self.temperature = self.config[self.intent]["model"]["temperature"]
        self.max_output_tokens = self.config[self.intent]["model"]["max_output_tokens"]

        parameters = dict(
            temperature=self.temperature, max_output_tokens=self.max_output_tokens
        )
        logger.debug(f"Model parameters: {parameters}")
        return parameters

    def get_prompt(self):
        prompt = self.config[self.intent]["prompts"][self.prompt_tag]
        logger.debug(f"Prompt Template: {prompt}")
        return prompt

    def get_llm_model(self):
        self.model_name = self.config[self.intent]["model"]["name"]
        llm = VertexAI(model_name=self.model_name)
        return llm

    def match_chroma(self, snippet: str, language: str):
        logger.info("Performing similarity search")
        language_filter = (
            {"languages": language.lower()} if language.lower() != "none" else {}
        )

        code_docs = self.chroma.similarity_search_with_relevance_scores(
            query=snippet, k=5, filter=language_filter
        )
        code_blobs = {}
        for code_doc in code_docs:
            blob_name = code_doc[0].metadata["filenames"]
            if not code_blobs.get(blob_name):
                code_blobs[blob_name] = (code_doc[0].page_content, code_doc[1])
            break
        response = []
        for filename, code_doc in code_blobs.items():
            response.append(
                {"snippet": code_doc[0], "source": filename, "score": code_doc[1]}
            )
        return response

    def perform_coding_task(self, text, **prompt_args):
        llm = self.get_llm_model()
        prompt = self.get_prompt()
        parameters = self.get_model_parameters()

        prefix = prompt.format(text=text, **prompt_args)
        logger.debug(f"Formatted Prompt: {prefix}")

        logger.info(f"Calling {self.model_name} model")
        result = llm.predict(prefix, **parameters)
        response = result

        result_object = AssistantResponse(
            input_query=text,
            intent=self.intent,
            prompt=prefix,
            model_name=self.model_name,
            model_parameters=parameters,
            response=response,
        )
        return result_object


In [ ]:
# Instantiate Pair Programming assistant with Chroma DB and Prompt Tag
pp_assistant = PairProgrammingAssistant(chroma=chroma, prompt_tag="zero_shot")

def process(input_: str, intent=None, **prompt_args):
    global pp_assistant
    logger.info(
        f"Inputs from Gradio UI: \nInput: {input_}, \n"
        f"Intent: {intent}, \nOther args: {prompt_args}"
    )
    result_object = pp_assistant.perform_coding_task(text=input_, **prompt_args)

    logger.debug(f"Result from LLM: {result_object.to_dict()}")

    bot_message = str(result_object.response)
    bot_message = "\n".join(bot_message.split("\n")[1:-1])
    return bot_message

def on_click_bug_fix(code_input, code_task):
    global pp_assistant

    logger.debug(
        f"Bug detection/fix suggestion initiated for: code_input: "
        f"{code_input}, code_task: {code_task}"
    )

    intent_mapping = {
        "Bug Detection": "bug_detection",
        "Fix Suggestion": "fix_suggestion",
    }
    intent = intent_mapping[code_task]
    pp_assistant.intent = intent

    language = pp_assistant.detect_language(code_input)
    logger.info(f"Detected language: {language}")

    logger.info("Performing Chroma search to fetch indexed code...")
    similar_docs = pp_assistant.match_chroma(snippet=code_input, language=language)
    rag_context = ""
    file_path = ""
    base_path = "/content/"
    if similar_docs:
        rag_context = similar_docs[0]["snippet"]
        file_path = base_path + similar_docs[0]["source"]

    prompt_args = {"context": rag_context}
    if language.lower() != "none":
        prompt_args = {"context": rag_context, "language": language}

    return (process(code_input, intent=intent, **prompt_args), file_path, rag_context)

import gradio as gr

bug_detection_example_input = """
def divide(a, b):
  return a // b
"""

fix_suggestion_example_input = "Fix the bug in the division function above to handle division by zero."

with gr.Blocks() as demo:
    with gr.Row():
        gr.Markdown("""# Bug Detection and Fix Suggestion Demo""")

    with gr.Tab("Bug Detection and Fix Suggestion Interface"):
        with gr.Row():
            code_task = gr.Dropdown(
                choices=["Bug Detection", "Fix Suggestion"],
                label="Code task",
                value="Bug Detection",
            )
        with gr.Row():
            with gr.Column(scale=1):
                code_input = gr.Code(label="Task", lines=20, scale=1, language="python")

            with gr.Column(scale=1):
                code_output = gr.Code(
                    label="Output", lines=20, scale=1, language="python"
                )

        with gr.Row() as metadata_block:
            with gr.Column(scale=1):
                metadata = gr.Code(label="RAG Context", lines=10, language="python")
                ref_file_path = gr.Textbox(
                    label="Reference File Path", show_copy_button=True
                )

        with gr.Row():
            with gr.Column(scale=1):
                submit = gr.Button(value="Run", variant="primary")
            with gr.Column(scale=1):
                reset = gr.Button(value="Reset", variant="secondary")

        with gr.Row():
            gr.Examples(
                [
                    ["Bug Detection", f"""{bug_detection_example_input}""", "", "", ""],
                    ["Fix Suggestion", f"{fix_suggestion_example_input}", "", "", ""],
                ],
                inputs=[code_task, code_input, code_output, metadata, ref_file_path],
            )

        submit.click(
            fn=on_click_bug_fix,
            inputs=[code_input, code_task],
            outputs=[code_output, ref_file_path, metadata],
        )
        reset.click(
            fn=lambda x: ("", "", "", ""),
            inputs=[code_input],
            outputs=[code_input, code_output, metadata, ref_file_path],
        )

demo.launch(share=True, debug=True, inbrowser=True)


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.41.2, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://c5911d561c4a8bcbbd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


2024-07-22 21:02:03.874 | DEBUG    | __main__:on_click_bug_fix:21 - Bug detection/fix suggestion initiated for: code_input: def divide(a, b):
    return a // b
, code_task: Bug Detection
2024-07-22 21:02:03.876 | INFO     | __main__:detect_language:45 - Language detection prompt: You are an elite programmer who knows syntax to multiple programming language. You are provided with a code snippet encapsulated by back-ticks (```). You are tasked to identify in what programming language out of python, java, golang the code snippet belongs to. Only give the name of the language and nothing else in the output. ``` {text} ```

/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
2024-07-22 21:02:04.588 | INFO     | __main__:on_click_bug_fix:34 - Detected language: python
2024-07-22 21:02:04.589

In [ ]:
logger.disable("__main__")
demo.launch(share=True, debug=True, inbrowser=True)
